<a href="https://colab.research.google.com/github/nicolaiberk/bild/blob/main/code/content_analysis/bert/mig/_mig_clsfr/mig_classifier_BERT_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT Model to classify German news articles

This is part of a project on migration attention in German news and their impact on issue attitudes and definitions, as well as political violence. The classifier indicated whether the content is about migration.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

In [3]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import TensorDataset, DataLoader

import os
os.environ["WANDB_DISABLED"] = "true"

# set random seed for reproducibility
SEED_GLOBAL = 42
np.random.seed(SEED_GLOBAL)

## Prepare data

In [4]:
# load classified migration data
print('Loading data...')
dta = pd.read_csv('https://www.dropbox.com/s/t47m1bppr3f1na5/handcoding_finished.csv?dl=1', sep = ',')
print('\tDone!')

Loading data...
	Done!


In [5]:
# define outcome
dta['mig_bin'] = dta.mig == 'Ja'

In [6]:
# shuffle data
dta = dta.sample(frac = 1, random_state=42).reset_index(drop = True)

In [7]:
#determine train-test-val split
splits = ['train']*1400
splits.extend(['test']*200)
splits.extend(['val']*200)
dta['split'] = splits

# Split dataset into pre-specified training, validation, and test sets
train = dta[['text', 'mig_bin']][dta['split']=='train'].reset_index(drop = True)
test = dta[['text', 'mig_bin']][dta['split']=='test'].reset_index(drop = True)
val = dta[['text', 'mig_bin']][dta['split']=='val'].reset_index(drop = True)

print(train.shape)
print(test.shape)
print(val.shape)


(1400, 2)
(200, 2)
(200, 2)


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [9]:
model_name = 'bert-base-german-cased'
BERT_tokenizer = AutoTokenizer.from_pretrained(model_name,
                                               model_max_length = 512)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/255k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/485k [00:00<?, ?B/s]

In [10]:
train_encodings = BERT_tokenizer(list(train['text']), padding='max_length', truncation=True, return_tensors='pt')
test_encodings = BERT_tokenizer(list(test['text']), padding='max_length', truncation=True, return_tensors='pt')
val_encodings = BERT_tokenizer(list(val['text']), padding='max_length', truncation=True, return_tensors='pt')

In [11]:
train_labels = [1 if label else 0 for label in train['mig_bin']]
test_labels = [1 if label else 0 for label in test['mig_bin']]
val_labels = [1 if label else 0 for label in val['mig_bin']]

In [12]:
import torch

class MigDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MigDataset(train_encodings, train_labels)
val_dataset = MigDataset(val_encodings, val_labels)
test_dataset = MigDataset(test_encodings, test_labels)

## Setup training, train

In [13]:
BERT_model = AutoModelForSequenceClassification.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
BERT_model = BERT_model.to(device)
train_encodings = train_encodings.to(device)
train_labels = torch.tensor(train_labels).to(device)
val_encodings = val_encodings.to(device)
val_labels = torch.tensor(val_labels).to(device)
test_encodings = test_encodings.to(device)
test_labels = torch.tensor(test_labels).to(device)

In [15]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [16]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    dataloader_pin_memory=False,
    evaluation_strategy="epoch",
    seed=SEED_GLOBAL
)

trainer = Trainer(
    model=BERT_model,                    # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [17]:
trainer.train()

<ipython-input-12-bcbef893af2f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.500546,0.740000,0.694118,0.595960,0.830986
2,0.585800,0.250903,0.915000,0.879433,0.885714,0.873239
3,0.204100,0.250655,0.915000,0.888889,0.829268,0.957746
4,0.150000,0.468108,0.885000,0.818898,0.928571,0.732394
5,0.111700,0.479099,0.910000,0.869565,0.895522,0.845070
6,0.059600,0.779605,0.885000,0.816000,0.944444,0.718310
7,0.048900,0.601063,0.905000,0.861314,0.893939,0.830986
8,0.011200,0.644053,0.900000,0.857143,0.869565,0.845070
9,0.011200,0.673874,0.910000,0.873239,0.873239,0.873239
10,0.000300,0.701374,0.910000,0.873239,0.873239,0.873239


<ipython-input-12-bcbef893af2f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-12-bcbef893af2f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-12-bcbef893af2f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-12-bcbef893af2f>:9: UserWarning: To copy construct from a 

TrainOutput(global_step=880, training_loss=0.13314731114713305, metrics={'train_runtime': 1409.4121, 'train_samples_per_second': 9.933, 'train_steps_per_second': 0.624, 'total_flos': 3683554775040000.0, 'train_loss': 0.13314731114713305, 'epoch': 10.0})

In [18]:
# save
trainer.save_model("drive/MyDrive/Bild/mig_clsfr_BERT_torch")

## Assess performance

In [19]:
trainer.evaluate(test_dataset)

<ipython-input-12-bcbef893af2f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.5635338425636292,
 'eval_accuracy': 0.93,
 'eval_f1': 0.9078947368421053,
 'eval_precision': 0.9078947368421053,
 'eval_recall': 0.9078947368421053,
 'eval_runtime': 6.062,
 'eval_samples_per_second': 32.992,
 'eval_steps_per_second': 0.66,
 'epoch': 10.0}